In [6]:
from sklearn.neighbors import KDTree
import numpy as np
import random
import dask.array as da
import napari
import pandas as pd
import zarr

%gui qt

coord_path = 'C:/Users/camer/OneDrive - Allen Institute/Documents/Unet/0539050069-0035_r.csv'

coord = np.loadtxt(coord_path, skiprows=1, delimiter=',')

In [27]:
z = zarr.open('C:/Users/camer/OneDrive - Allen Institute/Documents/Unet/0539050069-0035.zarr')
for i in range(z.shape[0] // 2048):
    z[(i*2048)-1:(i*2048)+1, :, 0] = 5000
for i in range (z.shape[1] // 2048):
    z[:, (i*2048)-1:(i*2048)+1, 0] = 5000
image = da.from_zarr(z)

In [28]:
# zarr_img_path = 'C:/Users/camer/OneDrive - Allen Institute/Documents/Unet/0539050069-0035.zarr'
# image = da.from_zarr(zarr_img_path)

In [29]:
coord_df = pd.DataFrame(coord)
coord_df.columns = ['y', 'x', 'size']
size_over_50 = coord_df['size'] >= 50
coord_df = coord_df[size_over_50]
distance_from_2048 = 50
before_vert = coord_df['x'] % 2048 <= distance_from_2048
after_vert = coord_df['x'] % 2048 >= (2048 - distance_from_2048)
before_horiz = coord_df['y'] % 2048 <= distance_from_2048
after_horiz = coord_df['y'] % 2048 >= (2048 - distance_from_2048)
coord_df = coord_df[(before_horiz | after_horiz) | (before_vert | after_vert)]
coord_s50 = coord_df.to_numpy()
print(coord_s50.shape)

(1433, 3)


In [30]:
kdt = KDTree(coord_s50[:,:2], leaf_size=30, metric='euclidean')

close_points = []
close_set =[]
indices_to_drop = []

close = kdt.query_radius(coord_s50[:,:2], r=15) 
for i in close:
    if i.size > 1:
        close_points.append(list(i))
for i in close_points:
    if not i in close_set:
        close_set.append(i)
print("Close points are: ", str(close_set))
for i in close_set:
    while len(i)>1:
        index = random.randrange(len(i))
        indices_to_drop.append(i[index])
        del i[index]

print(str(len(indices_to_drop)),"points to be removed:", str(indices_to_drop))
# print("These points are located at:")
# np.set_printoptions(suppress=True)
# for i in indices_to_drop:
#     print(coord[indices_to_drop])
# np.set_printoptions(suppress=False)
coord_s50_clean = np.array([i for j, i in enumerate(coord_s50) if j not in indices_to_drop])
coord_s50_removed = np.array([i for j, i in enumerate(coord_s50) if j in indices_to_drop])

print(str(coord_s50.shape[0]-coord_s50_clean.shape[0]), "points removed")

Close points are:  [[137, 172], [171, 322], [346, 185], [646, 887]]
4 points to be removed: [172, 322, 185, 646]
4 points removed


In [32]:
viewer = napari.view_image(image[:, :, 0], name='slice')
viewer.add_points(coord_s50_removed[:, :2], size=20, face_color='yellow')
viewer.add_points(coord_s50_clean[:, :2], size=10, face_color='red')

C:\Users\camer\anaconda3\envs\napari\lib\site-packages\napari\_vispy\vispy_image_layer.py:198: UserWarning: data shape (30000, 40000) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(


<Points layer 'Points [1]' at 0x17125b81b20>

In [276]:
np.savetxt('/Users/elyse.morin/OneDrive - Allen Institute/Transgenic_Technology_Elyse/Projects/Unet_test_output/547656/csv/0539050069-0035_r_clean.csv', coord_clean, delimiter=",")

np.savetxt('/Users/elyse.morin/OneDrive - Allen Institute/Transgenic_Technology_Elyse/Projects/Unet_test_output/547656/csv/0539050069-0035_r_removed.csv', coord_removed, delimiter=",")